<a href="https://colab.research.google.com/github/david-levin11/Verification_Notebooks/blob/main/Ensemble_PlanView_Plotter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This notebook will plot current and archived ensemble data in plan view format**
<br/>
Description--Currently this tool is only working for the ECMWF ensemble surface fields.  More development of this tool is underway so keep checking back in for updates.  If there is something that you would like to see added to this tool please contact me.

- David Levin, Arctic Testbed & Proving Ground, Anchorage Alaska

##**1 - Install and Import Packages**
This will take about a minute to run.

In [ ]:
# @title
!pip install ecmwflibs
!pip install eccodes
!pip install cfgrib
!pip install curl
!pip install wgrib2
!pip install herbie-data[extras]
!pip install requests

import os
from herbie import Herbie
from datetime import datetime, timedelta
import xarray as xr
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.ndimage as ndimage
from PIL import Image

##**2 - Select Options & Go!**
You can run this block multiple times without having to run step 1 again.

Graphics will save to your working directory (see the little folder icon on your left toolbar).

In [ ]:
#@markdown **Which ensemble system are we plotting?**
model = "GEFS" #@param ["EPS", "GEFS"]
#@markdown **Choose a model run date**
rundate = "2024-10-03" #@param {type: "date"}
#@markdown **Choose model run start hour (UTC)
runhour = 0 #@param{type: "slider", min:0, max:21, step:6}
#@markdown **Choose your forecast start hour (in hours from the model start time...ex: F06)**
start_hour = 48 #@param {type:"slider", min:0, max:240, step:1}
#@markdown **Choose your forecast end hour**
end_hour = 51 #@param {type:"slider", min:0, max:240, step:3}
#@markdown **Choose your plot type**
plot_type = "MSLPLowLocs" #@param ["----Surface----","MSLPLowLocs" ]
timestep = 3
extent = [-180, -130, 52, 70]
######################### Colormaps & Vars Config ##############################

mslcolors = [
    '#f096ee', '#f096ee', '#f091ee', '#f079ee', '#f079ee', '#f079ee',
    '#f079ee', '#ea59e5', '#ea59e5', '#ea59e5', '#ea59e5', '#e71add',
    '#e71add', '#e71add', '#e71add', '#c012b8', '#c012b8', '#c012b8',
    '#c012b8', '#bb11b3', '#a50f9d', '#a50f9d', '#a50f9d', '#a50f9d',
    '#800b78', '#800b78', '#800b78', '#800b78', '#680864', '#680864',
    '#680864', '#680864', '#590555', '#560552', '#560552', '#560552',
    '#560552', '#281aa7', '#281aa7', '#281aa7', '#281aa7', '#281aa7',
    '#281aa7', '#281aa7', '#281aa7', '#362aaf', '#362aaf', '#362aaf',
    '#362aaf', '#362aaf', '#4038c3', '#4038c3', '#4038c3', '#4038c3',
    '#6e63e1', '#6e63e1', '#6e63e1', '#6e63e1', '#988af9', '#a192ff',
    '#a192ff', '#a192ff', '#ada5ff', '#bab9ff', '#bab9ff', '#bab9ff',
    '#bab9ff', '#dddeff', '#dddeff', '#dddeff', '#dddeff', '#2066d5',
    '#2066d5', '#2066d5', '#2066d5', '#276fe8', '#2971ed', '#2971ed',
    '#2971ed', '#2971ed', '#3586f5', '#3586f5', '#3586f5', '#3586f5',
    '#59a6f7', '#59a6f7', '#59a6f7', '#59a6f7', '#7bbdfa', '#9dd4fd',
    '#9dd4fd', '#9dd4fd', '#a1d8fc', '#b9f0fa', '#b9f0fa', '#b9f0fa',
    '#b9f0fa', '#e2fdfd', '#e2fdfd', '#e2fdfd', '#e2fdfd', '#ceffbc',
    '#ceffbc', '#ceffbc', '#ceffbc', '#b7f9a1', '#a1f487', '#a1f487',
    '#a1f487', '#97f37b', '#6af040', '#6af040', '#6af040', '#6af040',
    '#3eb300', '#3eb300', '#3eb300', '#3eb300', '#329e00', '#329e00',
    '#329e00', '#329e00', '#98cc52', '#fffaa5', '#fffaa5', '#fffaa5',
    '#fffaa5', '#fee86d', '#fee86d', '#fee86d', '#fee86d', '#fbbf1e',
    '#fbbf1e', '#fbbf1e', '#fbbf1e', '#fab918', '#f99d00', '#f99d00',
    '#f99d00', '#f99d00', '#f75d00', '#f75d00', '#f75d00', '#f75d00',
    '#f62b00', '#f62b00', '#f62b00', '#f62b00', '#d61100', '#d61100',
    '#d61100', '#d61100', '#cb0e00', '#9a0300', '#9a0300', '#9a0300',
    '#9a0300', '#603c2d', '#603c2d', '#603c2d', '#603c2d', '#86664b',
    '#86664b', '#86664b', '#86664b', '#86664b', '#af8f7a', '#af8f7a',
    '#af8f7a', '#af8f7a', '#dac1b1', '#dac1b1', '#dac1b1', '#dac1b1',
    '#f4a29a', '#f4a29a', '#f4a29a', '#f4a29a', '#db6260', '#db6260',
    '#db6260', '#db6260', '#db6260', '#c33936', '#c33936', '#c33936',
    '#c33936', '#9e2312', '#9e2312', '#9e2312', '#9e2312', '#796360',
    '#727170', '#727170', '#727170', '#7e7e7d', '#8b8b8b', '#8b8b8b',
    '#8b8b8b', '#8b8b8b', '#ababab', '#ababab', '#ababab', '#ababab',
    '#c9cfc7', '#c9cfc7', '#c9cfc7', '#c9cfc7', '#dedfde', '#e3e3e3',
    '#e3e3e3'
]

# Dictionary mapping models to their respective variable names
model_variable_mapping = {
    "GEFS": {
        "MSLPLowLocs": "prmsl",
        "TotalPrecip": "tp"
    },
    "EPS": {
        "MSLPLowLocs": "msl",
        "TotalPrecip": "tp"
    }
}

model_cbar_mapping = {"MSLPLowLocs": [880, 1080, 4, 20]
}

######################### Functions & Methods #################################
def adjust_to_nearest_divisible_by_6(number):
    if number % 6 == 0:
        return number
    else:
        # Calculate the remainder when the number is divided by 3
        remainder = number % 6
        # Calculate the nearest number divisible by 3
        if remainder < 3:
            return number - 1
        elif remainder >= 3:
            return number + 1

def Pa_to_hPa(pascal):
  hPa = pascal/100
  return hPa

def unit_conversion(variable_name, data):
    if variable_name in ["prmsl", "msl"]:
        return Pa_to_hPa(data)  # Convert pressure to hPa
    elif variable_name == "tp":
        return data  # Total precipitation is already in mm or appropriate unit
    else:
        return data  # Default: no conversion

def indentify_lows(data):
  smoothed_mslp = ndimage.gaussian_filter(data, sigma=3)
  local_minima = (smoothed_mslp == ndimage.minimum_filter(smoothed_mslp, size=9))
  lows_y, lows_x = np.where(local_minima)
  return lows_x, lows_y

# Prepare the model data dynamically based on model and plot type
def prepare_data_for_plotting(model, plot_type, ensemble_ds, mean_ds):
    # Get the correct field name based on model and plot type
    variable_field = model_variable_mapping[model][plot_type]

    # Convert values if necessary
    ensemble_data = unit_conversion(variable_field, ensemble_ds[variable_field].values)
    mean_data = unit_conversion(variable_field, mean_ds[variable_field].values)
    # Handle coordinates
    #lons = np.where(ensemble_ds.longitude < 0, ensemble_ds.longitude + 360, ensemble_ds.longitude)
    lon2d, lat2d = np.meshgrid(ensemble_ds.longitude, ensemble_ds.latitude)
    #print(lon2d)
    #print(lat2d)
    #print(ensemble_data)
    #print(mean_data)
    return ensemble_data, mean_data, lon2d, lat2d

def get_colors(colorbar):
  # setting scales and colormaps
  myrange = np.arange(875, 1085, 5)
  cbar_range = np.arange(880, 1080, 20)
  contourrange = np.arange(880, 1080, 4)
  img = plt.imread(colorbar)
  # x and y positions of the rectangle centers are found via image editor
  ypos = 34
  xposstart = 33
  xposend = 1028
  valmin = 875
  valmax = 1085
  # extract the color from each of the rectangles
  colors = [matplotlib.colors.to_hex(img[ypos, int(xpos)]) for xpos in np.linspace(xposstart, xposend, valmax-valmin+1)]
  return colors

def get_gefs(rstring, fhour, field):
  members = []
  for i in range(0,31):
    print(f"Now working on member {i}")
    try:
      H = Herbie(rstring, model='gefs', product='atmos.25', member=i, fxx=fhour)
      ds = H.xarray(f"{field}")
      members.append(ds)
    except Exception as e:
      print(e)
      continue
  # now concatenating along the number axis
  gefs_ds = xr.concat(members, dim='number')
  return gefs_ds

def plot_low_loc(msl_all, lat2d, lon2d, ax, datacrs):
  # plotting low locs
  if model == "EPS":
    westbound = -180
    eastbound = -130
  if model == "GEFS":
    westbound = 180
    eastbound = 230
  for i in range(0, len(msl_all)):
      lows_x, lows_y = indentify_lows(msl_all[i])
      symbol = "$L$"
      for x, low in enumerate(lows_y):
          # plt.annotate doesn't have a transform option so we have to create it
          if westbound < lon2d[low, lows_x[x]] < eastbound and 52 < lat2d[low, lows_x[x]] < 70:

              #ax.scatter(lon2d[low, lows_x[x]], lat2d[low, lows_x[x]], s=35, marker=symbol, color='white', transform=datacrs)
              at_x, at_y = ax.projection.transform_point(lon2d[low, lows_x[x]], lat2d[low, lows_x[x]], src_crs=datacrs)
              text1 = 'L'
              text2 = int(msl_all[i][low,lows_x[x]])
              label1 = ax.annotate(text1, xy=(at_x, at_y), color='black', fontweight='bold', fontsize=8)
              label2 = ax.annotate(text2, xy=(at_x, at_y), xytext=(0,-10), textcoords='offset points', color='black', fontsize=6)

# Fetching data for different models and fields
def get_model_data(model, runstring, fcst_hr, field):
    if model == "EPS":
        H = Herbie(runstring, model='ifs', product='enfo', fxx=fcst_hr)
        return H.xarray(f"{field}")
    elif model == "GEFS":
        print(f"Forecast hour is: {fcst_hr} and runstring is {runstring} and field is {field}")
        return get_gefs(runstring, fcst_hr, field.upper())

# Setup plot configuration and projection
def setup_plot(extent):
    fig = plt.figure(figsize=[10, 10])
    datacrs = ccrs.PlateCarree()
    plotcrs = ccrs.NorthPolarStereo(true_scale_latitude=60, central_longitude=202.993)
    ax = plt.axes(projection=plotcrs)
    ax.set_extent(extent, datacrs)
    ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=1)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    return fig, ax, datacrs

# Colormap and range setup
def configure_colormap(colorlist, var):
    cmap = ListedColormap(colorlist)
    contourrange = np.arange(model_cbar_mapping[var][0], model_cbar_mapping[var][1], model_cbar_mapping[var][2])
    cbar_range = np.arange(model_cbar_mapping[var][0], model_cbar_mapping[var][1], model_cbar_mapping[var][3])
    return cmap, contourrange, cbar_range

# Add plot titles and labels
def set_plot_titles(ax, model, runhour, rundate, forecast_hours, valid_times, pos, time_valid):
    title = f"{model} | Init {runhour}Z {datetime.strptime(rundate, '%Y-%m-%d').strftime('%d %b %Y')} | Mean MSLP + Low Locations (hPa)"
    #ax.set_title(title, fontsize=9, weight='bold', loc='left', color='#272727')
    #ax.text(1.00, 1.017, f'Hour: {forecast_hours[model_valid_times[pos]]} | Valid: {time_valid.strftime("%HZ %d %b %Y")}', transform=ax.transAxes, fontsize=9, color='k', weight='semibold')
    ax.set_title(title,fontsize=9, weight='bold', stretch='normal', family='sans-serif', loc='left',color='#272727')
    plt.text(1.00, 1.017, f'Hour: {forecast_hours[valid_times[pos]]} | Valid: {time_valid.strftime("%HZ %d %b %Y")}', horizontalalignment='right',fontsize=9,color='k',transform = ax.transAxes, weight='semibold' )
    #adding copyright infor for the ECMWF data
    if model == "EPS":
      plt.text(1.00,0.01,f"Data Copyright © {datetime.strptime(rundate, '%Y-%m-%d').strftime('%Y')} European Centre for Medium-Range Weather Forecasts (ECMWF)",horizontalalignment='right',color='k',transform = ax.transAxes, fontsize=7, style="italic")

# Main plotting function
def plot_data(model, plot_type, ds_list, forecast_hours, valid_times, runstring, runhour, rundate, extent):
    for pos, ds in enumerate(ds_list):
        time_valid = datetime.strptime(runstring, "%Y-%m-%d %H:%M") + timedelta(hours=forecast_hours[pos])

        if model == "EPS":
          # Loop through all datasets in ds_list to find the one with 3 dimensions
          #EPS data comes in a list of datasets, one with all ens members and
          # another representing the mean
          ensemble_ds = None
          mean_ds = None
          for ensds in ds:
              if len(ensds.dims) == 3:
                  ensemble_ds = ensds
              else:
                  mean_ds = ensds
        elif model == "GEFS":
            ensemble_ds = ds
            mean_ds = ds.mean(dim='number')
        # Prepare data dynamically based on model and plot type
        field_data, mean_field_data, lon2d, lat2d = prepare_data_for_plotting(model, plot_type, ensemble_ds, mean_ds)

        # Set up projections and plot configurations
        fig, ax, datacrs = setup_plot(extent)

        if plot_type == "MSLPLowLocs":
            cmap, contourrange, cbar_range = configure_colormap(mslcolors, plot_type)
            cf = ax.contourf(lon2d, lat2d, mean_field_data, contourrange, cmap=cmap, transform=datacrs)
            ct = ax.contour(lon2d, lat2d, mean_field_data, contourrange, colors='k', linewidths=0.5, transform=datacrs)

            cbar = plt.colorbar(cf, orientation='horizontal', shrink=0.7, ticks=cbar_range, pad=0, aspect=40, drawedges=True)
            cbar.ax.tick_params(labelsize=10)
            ax.clabel(ct, fmt='%d')

            plot_low_loc(field_data, lat2d, lon2d, ax, datacrs)

            set_plot_titles(ax, model, runhour, rundate, forecast_hours, valid_times, pos, time_valid)
            graphicname = f"{model}_init_{runhour}z_{datetime.strptime(rundate, '%Y-%m-%d').strftime('%Y%m%d')}_F{forecast_hours[pos]}_MSLPLowLocs.png"
            plt.savefig(graphicname, bbox_inches='tight')
            print(f"Done with {graphicname}. Check working directory for the png (folder icon on the left toolbar)")
            plt.close()

######################## Main function calls #####################################

forecast_hours = list(range(start_hour, end_hour + 1, timestep))

#Getting our model initialization times based on inputs
runhour = str(runhour).zfill(2)
runstring = rundate + " " +runhour + ":00"
run_time = rundate + " " +runhour + "Z"
print(runstring)
print(run_time)

model_timesteps = []
model_valid_times = []

for step, fcst_hr in enumerate(forecast_hours):
  try:
    ds = get_model_data(model, runstring, fcst_hr, model_variable_mapping[model][plot_type])
    model_timesteps.append(ds)
    model_valid_times.append(step)
  except Exception as e:
    print(e)
    continue

#Plotting data
plot_data(model, plot_type, model_timesteps, forecast_hours, model_valid_times, runstring, runhour, rundate, extent)
